An implementation of encoded $\ket{A^2} = \ket{\sqrt{-iY}} = e^{-i \frac{\pi}{4}Y} \ket{+} = \ket{1}$ preparation in the Steane code using stim.

In particular, $A^2 = XH$, which will be used to implement the gate in stim.

Maybe easiest to just do $Y$ eigenstate, $\ket{+i}$?

In [3]:
from stim import Circuit
from noisy_circuits import noisy_steane_encoder,noisy_repetition_encoder,noisy_encoded_y
from decoders import steane_transversal_decoder,repetition_transversal_xdecoder

The logical $X$ outcome for the repetition-encoded ancilla block indicates which eigenstate the Hadamard test tells us it projected onto.

The logical $Y$ outcome for the Steane-encoded data block will indicate which eigenstate is observed.

If these two observables do not match, then there is a logical error on the output.

Below is a test of the circuit without errors, which should give us no errors and always result in selected trials.

In [4]:
def construct_noisy_short_circuit(perr:float,flag=True) -> Circuit:
    r"""
    Method for producing a noisy circuit that in the absence of errors
    will initialize and measure the logical Pauli-Y eigenstate |+i>.
    In this particular circuit, the logical qubit is encoded in a Steane code
    while the logical ancilla is encoded into a seven-qubit repetition code.

    In this case the repetition encoding is only 3 qubits but there is still
    a transversal CNOT to be performed.
    """

    noisy_circ = Circuit()

    steaneBlock = [0,1,2,3,4,5,6]
    repnBlock = [7,8,9]

    # initialize the Y-eigenstate: S^dagger|+>

    noisy_circ.append("RX",steaneBlock[0])
    noisy_circ.append("S_DAG",steaneBlock[0])
    
    noisy_circ.append("RX",repnBlock[0])

    noisy_circ.append("DEPOLARIZE1",[steaneBlock[0],repnBlock[0]],perr)

    noisy_circ = noisy_steane_encoder(noisy_circ,steaneBlock,perr)
    noisy_circ = noisy_repetition_encoder(noisy_circ,repnBlock,perr,flag) # include flag qubit in encoder


    if flag:
        noisy_circ = noisy_encoded_y(noisy_circ,[1,4,6],repnBlock[:-1],perr)

        noisy_circ.append("MX",repnBlock[:-1]) # readout transversal logical-X

    else:
        noisy_circ = noisy_encoded_y(noisy_circ,[1,4,6],repnBlock,perr)

        noisy_circ.append("MX",repnBlock) # readout transversal logical-X

    noisy_circ.append("MY",steaneBlock) # readout transversal logical-Y
        
    return noisy_circ

In [5]:
c = construct_noisy_short_circuit(0.1,flag=True)
c.diagram()

q0: -RX-S_DAG-DEPOLARIZE1(0.1)------------------@-DEPOLARIZE2(0.1)--------------------@-DEPOLARIZE2(0.1)----------------------X----------------DEPOLARIZE2(0.1)--------------------X----------------DEPOLARIZE2(0.1)---------------------------------------DEPOLARIZE1(0.1)------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------MY:rec[4]--
                                                 | |                                   | |                                     |                |                                   |                |
 q1: ----------RX---------------DEPOLARIZE1(0.1)-|-|------------------DEPOLARIZE1(0.1)-|-|----------------@-DEPOLARIZE2(0.1)---@----------------DEPOLARIZE2(0.1)--------------------|----------------|----------------@-DEPOLARIZE2(0.1)--------------------DEPOLARIZE1(0.1)-----------------------------------------------------------------------------------------------------------------------------------------------S_DAG-----DEPOLARIZE1(0.1)-----------------------------------X-DEPOLARIZE2(0.1)----------------------S----------------DEPOLARIZE1(0.1)-----------------------------------MY:rec[5]--
                                                 | |                                   | |                | |                                                                       |                |                | |                                                                                                                                                                                                                                                               | |
 q2: ----------RX---------------DEPOLARIZE1(0.1)-|-|------------------DEPOLARIZE1(0.1)-|-|----------------|-|------------------DEPOLARIZE1(0.1)-@----------------DEPOLARIZE2(0.1)---@----------------DEPOLARIZE2(0.1)-|-|---------------------------------@-DEPOLARIZE2(0.1)------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|-|---------------------------------------------------------------------------------------------------------MY:rec[6]--
                                                 | |                                   | |                | |                                   |                |                                                    | |                                 | |                                                                                                                                                                                                                           | |
 q3: ----------RX---------------DEPOLARIZE1(0.1)-|-|----------------@-DEPOLARIZE2(0.1)-|-|----------------|-|----------------@-DEPOLARIZE2(0.1)-|----------------|----------------@-DEPOLARIZE2(0.1)------------------|-|----------------DEPOLARIZE1(0.1)-|-|----------------DEPOLARIZE1(0.1)-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|-|---------------------------------------------------------------------------------------------------------MY:rec[7]--
                                                 | |                | |                | |                | |                | |                |                |                | |                                 | |                                 | |                                                                                                                                                                                                                           | |
 q4: -

In [14]:
error_rates = [0,
              1.0*10**(-5), 2.5*10**(-5),5.0*10**(-5),7.5*10**(-5),
              1.0*10**(-4), 2.5*10**(-4),5.0*10**(-4),7.5*10**(-4),
              1.0*10**(-3), 2.5*10**(-3),5.0*10**(-3),7.5*10**(-3),
              1.0*10**(-2)]

results = []

shots = 10_000_000

for perr in error_rates:
    selected = 0
    errors = 0
    
    noisy_circ = construct_noisy_short_circuit(perr)
    sampler = noisy_circ.compile_sampler()
    sample = sampler.sample(shots)

    for rec in sample:
        if not rec[0]: # post-select on False flag qubit
            mx = repetition_transversal_xdecoder(rec[1:4])
            my = steane_transversal_decoder(rec[4:])

            if not mx: # post-select on expected observable for Y_L =+1
                selected +=1
                errors += not(mx==my)
        
    print(str(perr) + ', ' + str([selected/shots,errors/selected]))
    print()

0, [1.0, 0.0]

1e-05, [0.9997009, 1.280382962544097e-05]

2.5e-05, [0.9992712, 2.7319910750955297e-05]

5e-05, [0.9985586, 6.249007319149822e-05]

7.500000000000001e-05, [0.997801, 9.500892462525093e-05]

0.0001, [0.9970492, 0.00012226076707147452]

0.00025, [0.992736, 0.00030290026754343553]

0.0005, [0.9854569, 0.0006390944139718338]

0.00075, [0.9784107, 0.00097034915910057]

0.001, [0.9712587, 0.0013436173081383981]

0.0025, [0.9309108, 0.003920354130599838]

0.005, [0.8693704, 0.009590733707979936]

0.0075, [0.8145862, 0.016899132344741417]

0.01, [0.7659855, 0.02574474320989105]



In [6]:
c = construct_noisy_short_circuit(0.1,flag=False)
c.diagram()

q0: -RX-S_DAG-DEPOLARIZE1(0.1)------------------@-DEPOLARIZE2(0.1)--------------------@-DEPOLARIZE2(0.1)----------------------X----------------DEPOLARIZE2(0.1)--------------------X----------------DEPOLARIZE2(0.1)---------------------------------------DEPOLARIZE1(0.1)----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------MY:rec[3]-
                                                | |                                   | |                                     |                |                                   |                |
q1: ----------RX---------------DEPOLARIZE1(0.1)-|-|------------------DEPOLARIZE1(0.1)-|-|----------------@-DEPOLARIZE2(0.1)---@----------------DEPOLARIZE2(0.1)--------------------|----------------|----------------@-DEPOLARIZE2(0.1)--------------------DEPOLARIZE1(0.1)--------------------------------------------------------S_DAG------------DEPOLARIZE1(0.1)-----------------------------------X-DEPOLARIZE2(0.1)----------------------S----------------DEPOLARIZE1(0.1)-----------------------------------MY:rec[4]-
                                                | |                                   | |                | |                                                                       |                |                | |                                                                                                                                                                               | |
q2: ----------RX---------------DEPOLARIZE1(0.1)-|-|------------------DEPOLARIZE1(0.1)-|-|----------------|-|------------------DEPOLARIZE1(0.1)-@----------------DEPOLARIZE2(0.1)---@----------------DEPOLARIZE2(0.1)-|-|---------------------------------@-DEPOLARIZE2(0.1)----------------------------------------------------------------------------------------------------------------------------|-|---------------------------------------------------------------------------------------------------------MY:rec[5]-
                                                | |                                   | |                | |                                   |                |                                                    | |                                 | |                                                                                                                                           | |
q3: ----------RX---------------DEPOLARIZE1(0.1)-|-|----------------@-DEPOLARIZE2(0.1)-|-|----------------|-|----------------@-DEPOLARIZE2(0.1)-|----------------|----------------@-DEPOLARIZE2(0.1)------------------|-|----------------DEPOLARIZE1(0.1)-|-|----------------DEPOLARIZE1(0.1)-----------------------------------------------------------------------------------------------------------|-|---------------------------------------------------------------------------------------------------------MY:rec[6]-
                                                | |                | |                | |                | |                | |                |                |                | |                                 | |                                 | |                                                                                                                                           | |
q4: ----------R----------------DEPOLARIZE1(0.1)-|-|----------------X-DEPOLARIZE2(0.1)-|-|----------------X-DEPOLARIZE2(0.1)-|-|----------------X----------------DEPOLARIZE2(0.1)-|-|---------------------------------|-|----------------DEPOLARIZE1(0.1)-|-|----------------DEPOLARIZE1(0.1)--------------------------------------------------------S_DAG------------DEPOLARIZE1(0.1)------------------|-|----------------X-DEPOLARIZE2(0.1)--------------------S----------------DEPOLARIZE1(0.1)------------------MY:rec[7]-
                                                | |                

In [8]:
error_rates = [0,
              1.0*10**(-5), 2.5*10**(-5),5.0*10**(-5),7.5*10**(-5),
              1.0*10**(-4), 2.5*10**(-4),5.0*10**(-4),7.5*10**(-4),
              1.0*10**(-3), 2.5*10**(-3),5.0*10**(-3),7.5*10**(-3),
              1.0*10**(-2)]

results = []

shots = 10_000_000

for perr in error_rates:
    selected = 0
    errors = 0
    
    noisy_circ = construct_noisy_short_circuit(perr,flag=False) # No flag qubit
    sampler = noisy_circ.compile_sampler()
    sample = sampler.sample(shots)

    for rec in sample:
        mx = repetition_transversal_xdecoder(rec[:3])
        my = steane_transversal_decoder(rec[3:])

        if not mx: # post-select on expected observable for Y_L =+1
            selected +=1
            errors += not(mx==my)
        
    print(str(perr) + ', ' + str([selected/shots,errors/selected]))
    print()

0, [1.0, 0.0]

1e-05, [0.9997721, 1.1402598652232843e-05]

2.5e-05, [0.999445, 2.691493779047371e-05]

5e-05, [0.9988617, 6.30717946238203e-05]

7.500000000000001e-05, [0.9983101, 9.215573397484409e-05]

0.0001, [0.9977459, 0.0001251821731364669]

0.00025, [0.9943865, 0.00031245396030617873]

0.0005, [0.9888893, 0.0006320222091593063]

0.00075, [0.9834083, 0.0009967375707526569]

0.001, [0.9779499, 0.0013350377151222163]

0.0025, [0.9467517, 0.003852224400547683]

0.005, [0.8987245, 0.00947809923953336]

0.0075, [0.8559075, 0.01669362635565175]

0.01, [0.8176802, 0.025451637449457623]

